In [1]:
import portfolio
import dataAck
import warnings
import numpy as np
warnings.filterwarnings("ignore")

allModels = portfolio.getModels("EWZ")
for model in allModels:
    print(model.describe())

((22, ('GDX', 32, None, None), 2, 'EWZ'), 22, 50)
((22, ('GDX', 32, None, None), 5, 'EWZ'), 22, 50)
((22, ('GDX', 32, None, None), 3, 'EWZ'), 22, 25)
((22, ('GDX', 32, None, None), 2, 'EWZ'), 22, 250)
((10, ('GDX', 32, None, None), 5, 'EWZ'), 10, 25)
((22, ('GDX', 32, None, None), 3, 'EWZ'), 22, 100)
((10, ('GDX', 32, None, None), 2, 'EWZ'), 10, 100)
((10, ('GDX', 32, None, None), 3, 'EWZ'), 10, 250)
((10, ('GDX', 32, None, None), 3, 'EWZ'), 10, 50)


# VIEW CORRELATION AND COVARIANCE

In [55]:
def generateAllReturns(allModels, joinedData):
    aggregateReturns = None
    for mod in allModels:
        print(mod.describe())
        algoReturn, factorReturn, predictions =  mod.makePredictions(joinedData) ##ONLY GET LAST 300 PREDICTIONS
        algoReturn.columns = [str(mod.describe())]
        if aggregateReturns is None:
            aggregateReturns = algoReturn
        else:
            aggregateReturns = aggregateReturns.join(algoReturn)
    return aggregateReturns

In [56]:
tickersRequired = []
tickersTraded = []
for mod in allModels:
    print(mod.describe())
    if mod.inputSeries.targetTicker not in tickersRequired:
        tickersRequired.append(mod.inputSeries.targetTicker)
    if mod.inputSeries.series.ticker not in tickersRequired:
        tickersRequired.append(mod.inputSeries.series.ticker)
    if mod.inputSeries.targetTicker not in tickersTraded:
        tickersTraded.append(mod.inputSeries.targetTicker)
    

pulledData, validTickers = dataAck.downloadTickerData(tickersRequired)

joinedData = dataAck.joinDatasets([pulledData[ticker] for ticker in pulledData])
modelReturns = generateAllReturns(allModels, joinedData[:-252])

((22, ('GDX', 32, None, None), 2, 'EWZ'), 22, 50)
((22, ('GDX', 32, None, None), 5, 'EWZ'), 22, 50)
((22, ('GDX', 32, None, None), 3, 'EWZ'), 22, 25)
((22, ('GDX', 32, None, None), 2, 'EWZ'), 22, 250)
((10, ('GDX', 32, None, None), 5, 'EWZ'), 10, 25)
((22, ('GDX', 32, None, None), 3, 'EWZ'), 22, 100)
((10, ('GDX', 32, None, None), 2, 'EWZ'), 10, 100)
((10, ('GDX', 32, None, None), 3, 'EWZ'), 10, 250)
((10, ('GDX', 32, None, None), 3, 'EWZ'), 10, 50)
GDX
EWZ
((22, ('GDX', 32, None, None), 2, 'EWZ'), 22, 50)


Process ForkProcess-53:
Process ForkProcess-36:
Process ForkProcess-48:
Process ForkProcess-63:
Process ForkProcess-45:
Process ForkProcess-20:
Process ForkProcess-34:
Process ForkProcess-31:
Process ForkProcess-47:
Process ForkProcess-13:
Process ForkProcess-40:
Process ForkProcess-22:
Process ForkProcess-30:
Traceback (most recent call last):
  File "/Users/patrickogrady/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Process ForkProcess-44:
Process ForkProcess-27:
Process ForkProcess-21:
Process ForkProcess-5:
  File "/Users/patrickogrady/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
Process ForkProcess-26:
Process ForkProcess-42:
Traceback (most recent call last):
Process ForkProcess-23:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkProcess-55:
  File "/Users/patrickogrady/anaconda3/lib/python3.6/mult

# STORE MODELS AS PORTFOLIO

In [36]:
import params
from google.cloud import datastore, storage, logging
import time
import pickle
import hashlib
import sys
import numpy as np
def storePortfolio(models, description):
    allHashes = []
    for model in models:
        organismHash = hashlib.sha224(str(model.describe()).encode('utf-8')).hexdigest()
        allHashes.append(organismHash)
    
    allHashes.sort()
    portfolioHash = hashlib.sha224(str(allHashes).encode('utf-8')).hexdigest()
    print("PORTFOLIO HASH:", portfolioHash)
    for hashing in allHashes:
        print(hashing)
        
    
        ##UPLOAD ORGANISM OBJECT
        while True:
            try:
                toUpload = {
                    "portfolio":portfolioHash,
                    "model":hashing
                }
                datastoreClient = datastore.Client('money-maker-1236')
                #HASH DIGEST
                key = datastoreClient.key(params.portfolioDB, hashlib.sha224(str(hashing + portfolioHash).encode('utf-8')).hexdigest()) #NEED TO HASH TO ENSURE UNDER COUNT
                organismToStore = datastore.Entity(key=key)
                organismToStore.update(toUpload)
                datastoreClient.put(organismToStore)
                break
            except:
                print("UPLOAD ERROR:", str(sys.exc_info()))
                time.sleep(10)
    
    while True:
        try:
            toUpload = {
                "description":description
            }
            datastoreClient = datastore.Client('money-maker-1236')
            #HASH DIGEST
            key = datastoreClient.key(params.portfolioLookup, portfolioHash) #NEED TO HASH TO ENSURE UNDER COUNT
            organismToStore = datastore.Entity(key=key)
            organismToStore.update(toUpload)
            datastoreClient.put(organismToStore)
            break
        except:
            print("UPLOAD ERROR:", str(sys.exc_info()))
            time.sleep(10)

In [38]:
storePortfolio(allModels, "EWZ 08-11-19")

PORTFOLIO HASH: 514d5ee998b05536888778bb1d1d48d638992056a6ed8bdb2023cee2
30e79488fb925fedbb3f63f6abb86757c4763d92bc4e893a87556d59
419d1aadd49f8b53c9f732b304c4d41a0576d4c721a3da0c0e0c7730
584efc9e68aed8795e0138754b77cc3c760d3454153bddf090caf8f9
7ff98f4707692aaf28e9b72000b57a7934e07a7d715957e4e389bff2
bad9f9f22f779a70cccb3c9197c9f0b5229ce68f4c648aa04e04f97f
bafabbb2ddbb6e294ed821e1e1e41116981d49addb415b95798bb67e
beef2ac53b15ac017d4a6c61d83d1a8126ca82d4025fbfbd582fb8df
c26a1518a8eb9d58caca155d4bb0dfbb0a52574d37e69cd3eb48282e
e1111d514a667ab22d8f48a2734a7dab0be1350f8ce70ca3b8a172ac


In [42]:
def getPortfolios():
    while True:
        try:
            datastore_client = datastore.Client('money-maker-1236')
            query = datastore_client.query(kind=params.portfolioLookup)
            retrievedPortfolios = [{
                "key":item.key.name,
                "description":item["description"]
            } for item in list(query.fetch())]

            return retrievedPortfolios
        except:
            time.sleep(10)
            print("DATA SOURCE RETRIEVAL ERROR:", str(sys.exc_info()))

In [44]:
portfolios = getPortfolios()
print(portfolios)

[{'key': '514d5ee998b05536888778bb1d1d48d638992056a6ed8bdb2023cee2', 'description': 'EWZ 08-11-19'}]


In [45]:
def getPortfolioModels(portfolioKey):
    while True:
        try:
            datastore_client = datastore.Client('money-maker-1236')
            query = datastore_client.query(kind=params.portfolioDB)
            query.add_filter('portfolio', '=', portfolioKey)
            
            retrievedModels = [item["model"] for item in list(query.fetch())]

            return retrievedModels
        except:
            time.sleep(10)
            print("DATA SOURCE RETRIEVAL ERROR:", str(sys.exc_info()))

In [47]:
for item in portfolios:
    print(item)
    print(getPortfolioModels(item["key"]))

{'key': '514d5ee998b05536888778bb1d1d48d638992056a6ed8bdb2023cee2', 'description': 'EWZ 08-11-19'}
['bafabbb2ddbb6e294ed821e1e1e41116981d49addb415b95798bb67e', 'e1111d514a667ab22d8f48a2734a7dab0be1350f8ce70ca3b8a172ac', 'beef2ac53b15ac017d4a6c61d83d1a8126ca82d4025fbfbd582fb8df', 'bad9f9f22f779a70cccb3c9197c9f0b5229ce68f4c648aa04e04f97f', '30e79488fb925fedbb3f63f6abb86757c4763d92bc4e893a87556d59', '419d1aadd49f8b53c9f732b304c4d41a0576d4c721a3da0c0e0c7730', '584efc9e68aed8795e0138754b77cc3c760d3454153bddf090caf8f9', 'c26a1518a8eb9d58caca155d4bb0dfbb0a52574d37e69cd3eb48282e', '7ff98f4707692aaf28e9b72000b57a7934e07a7d715957e4e389bff2']


In [52]:
def getModelsByKey(modelHashes):
    while True:
        try:
            datastore_client = datastore.Client('money-maker-1236')
            ##form keys
            keys = []
            for hashing in modelHashes:
                key = datastore_client.key(params.datastoreName, hashing)
                keys.append(key)
                
            retrievedModels = datastore_client.get_multi(keys)
            toReturn = []
            for source in retrievedModels:
                toReturn.append(pickle.loads(source["model"]))
            return toReturn
            
        except:
            time.sleep(10)
            print("DATA SOURCE RETRIEVAL ERROR:", str(sys.exc_info()))

In [53]:
getModelsByKey(getPortfolioModels(item["key"]))